# TUIA - Procesamiento de Lenguaje Natural - Unidad 7 - Cuaderno de práctica
**Unidad 7 - Agentes Autónomos y Sistemas Inteligentes**
<br>
<br>
**Docente teoría**:<br>
Teoría: MANSON, Juan Pablo    jpmanson@gmail.com [@juanpablomanson](https://twitter.com/juanpablomanson)<br>
[LinkedIN](https://www.linkedin.com/in/juanpablomanson/)<br><br>

## Instalación de Ollama

In [ ]:
# Descarga de Ollama
!curl -fsSL https://ollama.com/install.sh | sh

# Iniciamos Ollama en background
!rm -f ollama_start.sh
!echo '#!/bin/bash' > ollama_start.sh
!echo 'ollama serve' >> ollama_start.sh
# Make the script executable
!chmod +x ollama_start.sh
!nohup ./ollama_start.sh &

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
nohup: appending output to 'nohup.out'


Descarga del modelo LLM desde Ollama

In [ ]:
LLM_MODEL = "gemma3:12b-it-qat"
# LLM_MODEL = "phi3:medium"
# LLM_MODEL = "phi3:mini"
# LLM_MODEL = "phi4"
# LLM_MODEL = "qwen2.5"
# LLM_MODEL = "qwen2.5-coder"
!ollama pull $LLM_MODEL > ollama.log

Probamos que Ollama ya tenga disponible el modelo elegido descargado

In [ ]:
!ollama list

NAME                 ID              SIZE      MODIFIED               
gemma3:12b-it-qat    5d4fa005e7bb    8.9 GB    Less than a second ago    


Creamos un proxy, para poder usar Ollama con la misma interfaz API de OpenAI

In [ ]:
%%capture
!pip install litellm[proxy]
!nohup litellm --model ollama/$LLM_MODEL --port 8000 > litellm.log 2>&1 &

## Pruebas de razonamiento

### Prompts con Zephyr

In [ ]:
%%capture
!pip install python-decouple jinja2 llm-templates

In [ ]:
# !pip install python-decouple jinja2 llm-templates
# Utilizamos python-decouple para guardar las API keys en un archivo .env

from llm_templates import Formatter, Conversation
import requests
import json
from decouple import config
from typing import Any, Dict, List
from google.colab import userdata


# Función para aplicar el template de chat usando la librería llm-templates
def zephyr_chat_template(messages, add_generation_prompt=True):
    formatter = Formatter()
    conversation = Conversation(model='zephyr', messages=messages)
    # Renderizar la plantilla con los mensajes proporcionados
    return formatter.render(conversation, add_assistant_prompt=add_generation_prompt)

# Función de apoyo para mejorar la visualización en Colab
def word_wrap(text, max_words_per_line):
    words = text.split()
    lines = []
    current_line = []

    for word in words:
        current_line.append(word)
        if len(current_line) >= max_words_per_line:
            lines.append(' '.join(current_line))
            current_line = []

    # Agregar cualquier palabra restante
    if current_line:
        lines.append(' '.join(current_line))

    return '\n'.join(lines)

# Aquí hacemos la llamada el modelo
def generate_answer(prompt: str, max_new_tokens: int = 768) -> None:
    messages: List[Dict[str, str]] = [
        {
            "role": "system",
            "content": "Eres un asistente útil que siempre responde con respuestas veraces, útiles y basadas en hechos.",
        },
        {"role": "user", "content": prompt},
    ]

    try:
        prompt_formatted: str = zephyr_chat_template(messages, add_generation_prompt=True)

        # Tu clave API de Hugging Face
        api_key = config('HF_TOKEN', userdata.get('HF_TOKEN'))

        # URL de la API de Hugging Face para la generación de texto
        api_url = "https://api-inference.huggingface.co/models/HuggingFaceH4/zephyr-7b-beta"

        # Cabeceras para la solicitud
        headers = {"Authorization": f"Bearer {api_key}"}

        # Datos para enviar en la solicitud POST
				# Sobre los parámetros: https://huggingface.co/docs/transformers/main_classes/text_generation
        data = {
            "inputs": prompt_formatted,
            "parameters": {
                "max_new_tokens": max_new_tokens,
                "temperature": 0.7,
                "top_k": 50,
                "top_p": 0.95
            }
        }

        # Realizamos la solicitud POST
        response = requests.post(api_url, headers=headers, json=data)

        # Extraer respuesta
        respuesta = response.json()[0]["generated_text"][len(prompt_formatted):]
        return respuesta

    except Exception as e:
        print(f"An error occurred: {e}")


def solve_questions(questions):
  for idx, pregunta in enumerate(questions):
    print('PREGUNTA: ', word_wrap(pregunta, 24))
    print('\nRESPUESTA: ', word_wrap(generate_answer(pregunta), 24))
    print('--------------------------------------------')

In [ ]:

preguntas_de_razonamiento = [
    "Si Newton no hubiera formulado las leyes del movimiento, ¿cómo podría ser diferente la física moderna?",
    "Considera un coche autónomo con un sistema de frenos defectuoso en una situación donde debe elegir entre atropellar a un peatón o hacer una maniobra y dañar a sus pasajeros. ¿Qué debería hacer y por qué?",
    "¿En qué se parece el rol de una membrana celular en una célula biológica al rol del personal de seguridad en un aeropuerto?"
]

solve_questions(preguntas_de_razonamiento)

preguntas_de_logica = [
    "Todos los pájaros ponen huevos. Un cisne es un pájaro. ¿Se sigue lógicamente que un cisne pone huevos? ¿Por qué sí o por qué no?",
    "Si alguien argumenta que 'no deberíamos escuchar la opinión de una persona sobre el cambio climático porque no es científico', ¿qué falacia lógica está cometiendo?"
]

solve_questions(preguntas_de_logica)

preguntas_de_matematicas = [
    "Demuestra que para cada número primo \( p \), existe un número primo \( q \) tal que \( p \) no es igual a \( q \) y \( p \) divide \( q - 1 \).",
    "Una prueba médica para una enfermedad tiene una tasa de precisión del 95% para detectar la enfermedad cuando está presente (tasa de verdadero positivo) y una tasa de precisión del 90% para identificar correctamente cuando la enfermedad no está presente (tasa de verdadero negativo). Si el 1% de la población realmente tiene la enfermedad, ¿cuál es la probabilidad de que una persona tenga la enfermedad dado que dio positivo?",
    "Explica la paradoja de Banach–Tarski y sus implicaciones para el concepto de volumen en matemáticas."
]

solve_questions(preguntas_de_matematicas)

preguntas_de_programacion = [
    "Escribe un algoritmo eficiente para resolver el problema del viajante para un pequeño número de ciudades y explica por qué es eficiente.",
    "Explica la diferencia entre una copia profunda y una copia superficial de una estructura de datos. Proporciona un ejemplo en Python donde no entender esta diferencia podría llevar a un error."
]

solve_questions(preguntas_de_programacion)

preguntas_adversarias = [
    "Si doblas un pedazo de papel 42 veces, ¿llegará a la luna? Explica por qué esto es o no es posible.",
    "Si se tarda 10 minutos en cocinar un panqueque en una plancha que solo cabe un panqueque, ¿cuánto tiempo se tardaría en cocinar 10 panqueques?",
    "El pollo está listo para comer. Explica la ambigüedad en esta oración."
]

solve_questions(preguntas_adversarias)

PREGUNTA:  Si Newton no hubiera formulado las leyes del movimiento, ¿cómo podría ser diferente la física moderna?

RESPUESTA:  Si Newton no hubiera formulado las leyes del movimiento, la física moderna sería considerablemente diferente. Antes de Newton, los científicos y filósofos habían propuesto
diferentes teorías sobre el movimiento, como las de Aristóteles y Galileo. Sin embargo, estas teorías eran imprecisas y no podían explicar todos los fenómenos
del movimiento. Newton's Laws of Motion, publicadas en 1687, proporcionaron una base sólida para la física del movimiento. Estas leyes describen cómo los objetos
se mueven en respuesta a las fuerzas que actúan sobre ellas, y son esenciales para la comprensión de muchas áreas de la física moderna,
como la dinámica, la mecánica del sólido y la mecánica del fluido. Sin las Leyes de Newton, la física moderna tendría que recurrir a
teorías menos precisas y menos completas para explicar el comportamiento del movimiento. Además, la ciencia de 

In [ ]:
# Ejemplo de Zero Shot Chain of Thought (Zero-shot-CoT)
zero_shot_cot = "Fui al mercado y compré 10 manzanas. Le di 2 manzanas al vecino y 2 al reparador. Luego fui y compré 5 manzanas más y me comí 1."
zero_shot_cot += "\n¿Con cuántas manzanas me quedé?. Pensemos paso a paso."
print('PREGUNTA: ', zero_shot_cot)
print('\nRESPUESTA: ', generate_answer(zero_shot_cot))

PREGUNTA:  Fui al mercado y compré 10 manzanas. Le di 2 manzanas al vecino y 2 al reparador. Luego fui y compré 5 manzanas más y me comí 1.
¿Con cuántas manzanas me quedé?. Pensemos paso a paso.

RESPUESTA:  Al principio comprabas 10 manzanas, entonces tenías 10 manzanas. Luego dio 2 manzanas al vecino y 2 al reparador, por lo que se quedaron con 6 manzanas. Luego volviste a comprar 5 manzanas más, por lo que en total tienes 11 manzanas (6 de antes y 5 nuevas). Finalmente comiste una manzana, por lo que se quedaron con 10 manzanas (11 menos 1). Así que al final, se quedaste con 10 manzanas.

Pasos:
1. Compraste 10 manzanas.
2. Dije 2 manzanas al vecino y 2 al reparador.
3. Se quedaste con 6 manzanas.
4. Volviste a comprar 5 manzanas más.
5. Comiste una manzana.
6. Se quedaste con 10 manzanas.

Cantidad final: 10 manzanas.


In [ ]:
def compare_reasoning_approaches():
    # Problema base para comparar
    problem = "Una caja contiene 24 chocolates. Tres amigos deciden compartirlos de manera que el segundo recibe el doble que el primero, y el tercero recibe el triple que el primero. ¿Cuántos chocolates recibe cada uno?"
    print(f'Problema: {problem}')

    # Versión directa (sin CoT)
    direct_prompt = problem + "\n¿Cuántos chocolates recibe cada amigo?"

    # Versión CoT con ejemplos
    cot_prompt = f"""
    Veamos cómo resolver problemas de reparto paso a paso:

    Problema: En una fiesta hay 18 galletas y se reparten entre dos hermanos, de forma que uno recibe el doble que el otro. ¿Cuántas galletas recibe cada uno?
    Pensamiento:
    1. Si uno recibe el doble que el otro, podemos pensar en "partes"
    2. Si el primero recibe 1 parte, el segundo recibe 2 partes
    3. En total hay 3 partes
    4. 18 galletas ÷ 3 partes = 6 galletas por parte
    5. Primer hermano: 1 parte = 6 galletas
    6. Segundo hermano: 2 partes = 12 galletas
    Respuesta: El primer hermano recibe 6 galletas y el segundo 12 galletas

    Ahora resolvamos este problema:
    {problem}
    Pensamiento:
    """

    # Versión Zero-shot CoT
    zero_shot_prompt = problem + "\nResolvamos esto paso a paso:"

    print("\nComparación de enfoques de razonamiento:")
    print("\n1. Prompt Directo:")
    print(word_wrap(generate_answer(direct_prompt), 24))

    print("\n2. Chain of Thought con ejemplos:")
    print(word_wrap(generate_answer(cot_prompt), 24))

    print("\n3. Zero-shot Chain of Thought:")
    print(word_wrap(generate_answer(zero_shot_prompt), 24))

# Ejecutar la comparación
compare_reasoning_approaches()

Problema: Una caja contiene 24 chocolates. Tres amigos deciden compartirlos de manera que el segundo recibe el doble que el primero, y el tercero recibe el triple que el primero. ¿Cuántos chocolates recibe cada uno?

Comparación de enfoques de razonamiento:

1. Prompt Directo:
Para resolver este problema, podemos usar la ecuación: Segundo amigo recibe doble que el primero: Segundo amigo = Primero × 2 Tercero amigo recibe
triple que el primero: Tercero amigo = Primero × 3 Por supuesto, la suma total de los chocolates debe ser igual a la cantidad
inicial de 24: Total = Primero + Segundo + Tercero 24 = Primero + 2 × Primero + 3 × Primero 24 = 6
× Primero Primero = 4 Sustituimos el valor de Primero en las otras ecuaciones: Segundo amigo = 8 Tercero amigo = 12 Así que
cada amigo recibe: - El primero recibe 4 chocolates. - El segundo recibe 8 chocolates. - El tercero recibe 12 chocolates. Espero que esto
ayude!

2. Chain of Thought con ejemplos:
1. Si el primero recibe una parte, el segundo 

### Prompts con LLM local (Ollama)

In [ ]:
%%capture
!pip install ollama

In [ ]:
import ollama
from typing import List, Dict

def word_wrap(text, max_words_per_line):
    words = text.split()
    lines = []
    current_line = []

    for word in words:
        current_line.append(word)
        if len(current_line) >= max_words_per_line:
            lines.append(' '.join(current_line))
            current_line = []

    if current_line:
        lines.append(' '.join(current_line))

    return '\n'.join(lines)

def generate_answer(prompt: str, max_tokens: int = 768) -> str:
    try:
        # Creamos el mensaje con el sistema y el prompt del usuario
        messages = [
            {
                "role": "system",
                "content": "Eres un asistente útil que siempre responde con respuestas veraces, útiles y basadas en hechos."
            },
            {
                "role": "user",
                "content": prompt
            }
        ]

        # Llamamos a phi-3 usando ollama
        response = ollama.chat(
            model=LLM_MODEL,
            messages=messages,
            options={
                'num_predict': max_tokens,
                'temperature': 0.1
            }
        )

        # Extraemos la respuesta
        return response['message']['content']

    except Exception as e:
        return f"Ocurrió un error: {e}"

def solve_questions(questions):
    for idx, pregunta in enumerate(questions):
        print('PREGUNTA: ', word_wrap(pregunta, 24))
        print('\nRESPUESTA: ', word_wrap(generate_answer(pregunta), 24))
        print('--------------------------------------------')


In [ ]:
# Ejemplo de Zero Shot Chain of Thought (Zero-shot-CoT)
zero_shot_cot = "Fui al mercado y compré 10 manzanas. Le di 2 manzanas al vecino y 2 al reparador. Luego fui y compré 5 manzanas más y me comí 1."
zero_shot_cot += "\n¿Con cuántas manzanas me quedé?. Pensemos paso a paso."
print('PREGUNTA: ', zero_shot_cot)
print('\nRESPUESTA: ', generate_answer(zero_shot_cot))

PREGUNTA:  Fui al mercado y compré 10 manzanas. Le di 2 manzanas al vecino y 2 al reparador. Luego fui y compré 5 manzanas más y me comí 1.
¿Con cuántas manzanas me quedé?. Pensemos paso a paso.

RESPUESTA:  Claro, vamos a resolverlo paso a paso:

1.  **Comienzas con:** 10 manzanas.
2.  **Das al vecino:** 10 - 2 = 8 manzanas.
3.  **Das al reparador:** 8 - 2 = 6 manzanas.
4.  **Compras más:** 6 + 5 = 11 manzanas.
5.  **Te comes una:** 11 - 1 = 10 manzanas.

**Respuesta:** Te quedaste con 10 manzanas.
</end_of_turn>


In [ ]:
def compare_reasoning_approaches():
    # Problema base para comparar
    problem = "Una caja contiene 24 chocolates. Tres amigos deciden compartirlos de manera que el segundo recibe el doble que el primero, y el tercero recibe el triple que el primero. ¿Cuántos chocolates recibe cada uno?"
    print(f'Problema: {problem}')

    # Versión directa (sin CoT)
    direct_prompt = problem + "\n¿Cuántos chocolates recibe cada amigo?"

    # Versión CoT con ejemplos
    cot_prompt = f"""
    Veamos cómo resolver problemas de reparto paso a paso:

    Problema: En una fiesta hay 18 galletas y se reparten entre dos hermanos, de forma que uno recibe el doble que el otro. ¿Cuántas galletas recibe cada uno?
    Pensamiento:
    1. Si uno recibe el doble que el otro, podemos pensar en "partes"
    2. Si el primero recibe 1 parte, el segundo recibe 2 partes
    3. En total hay 3 partes
    4. 18 galletas ÷ 3 partes = 6 galletas por parte
    5. Primer hermano: 1 parte = 6 galletas
    6. Segundo hermano: 2 partes = 12 galletas
    Respuesta: El primer hermano recibe 6 galletas y el segundo 12 galletas

    Ahora resolvamos este problema:
    {problem}
    Pensamiento:
    """

    # Versión Zero-shot CoT
    zero_shot_prompt = problem + "\nResolvamos esto paso a paso:"

    print("Comparación de enfoques de razonamiento:")
    print("\n1. Prompt Directo:")
    print(word_wrap(generate_answer(direct_prompt), 24))

    print("\n2. Chain of Thought con ejemplos:")
    print(word_wrap(generate_answer(cot_prompt), 24))

    print("\n3. Zero-shot Chain of Thought:")
    print(word_wrap(generate_answer(zero_shot_prompt), 24))

# Ejecutar la comparación
compare_reasoning_approaches()

Problema: Una caja contiene 24 chocolates. Tres amigos deciden compartirlos de manera que el segundo recibe el doble que el primero, y el tercero recibe el triple que el primero. ¿Cuántos chocolates recibe cada uno?
Comparación de enfoques de razonamiento:

1. Prompt Directo:
Aquí está la solución al problema: * Sea 'x' la cantidad de chocolates que recibe el primer amigo. * El segundo amigo recibe 2x
chocolates. * El tercer amigo recibe 3x chocolates. La suma de los chocolates que reciben los tres amigos es igual al número total de
chocolates en la caja: x + 2x + 3x = 24 Combinando los términos, obtenemos: 6x = 24 Dividiendo ambos lados por 6, obtenemos:
x = 4 Por lo tanto: * El primer amigo recibe 4 chocolates. * El segundo amigo recibe 2 * 4 = 8 chocolates.
* El tercer amigo recibe 3 * 4 = 12 chocolates. **Respuesta:** El primer amigo recibe 4 chocolates, el segundo amigo recibe 8 chocolates
y el tercer amigo recibe 12 chocolates.

2. Chain of Thought con ejemplos:
Pensamiento: 1. S

## ReAct con herramientas

In [ ]:
%%capture
!pip install duckduckgo_search wikipedia beautifulsoup4 ollama

In [ ]:
import datetime
import wikipedia
import requests
import logging
import ollama
from typing import Dict, List, Tuple

# --- Configuración del Logging ---
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

# Nota: LLM_MODEL debe estar definido en otra parte del código
#LLM_MODEL = "gemma3:12b-it-qat"

# --- Definición de la Clase de Herramientas ---
class Herramientas:
    """Encapsula todas las funciones que el agente puede llamar."""
    def buscar_wikipedia(self, query: str) -> str:
        """Busca un resumen sobre un tema en Wikipedia en español."""
        logger.info(f"Ejecutando buscar_wikipedia con query: '{query}'")
        try:
            wikipedia.set_lang('es')
            # auto_suggest=False para evitar que Wikipedia cambie la query
            result = wikipedia.summary(query, sentences=3, auto_suggest=False)
            return result
        except wikipedia.exceptions.PageError:
            return f"Error: No se encontró la página para '{query}'. Intenta con otro término."
        except wikipedia.exceptions.DisambiguationError as e:
            return f"Error: La búsqueda '{query}' es ambigua. Opciones: {e.options[:3]}"
        except Exception as e:
            return f"Error inesperado en Wikipedia: {e}"

    def obtener_chiste(self, **kwargs) -> str:
        """Obtiene un chiste al azar en español."""
        logger.info("Ejecutando obtener_chiste")
        try:
            response = requests.get("https://v2.jokeapi.dev/joke/Any?lang=es&type=single,twopart")
            response.raise_for_status()
            data = response.json()
            if data.get("error"):
                return "Error: La API no pudo devolver un chiste en español."
            if data["type"] == "twopart":
                return f"Hemos obtenido este chiste: \"{data['setup']}... {data['delivery']}\""
            else:
                return f"Hemos obtenido este chiste: \"{data['joke']}\""
        except Exception as e:
            return f"Error inesperado en la API de chistes: {e}"

    def calcular_fecha(self, **kwargs) -> str:
        """Devuelve la fecha actual."""
        logger.info("Ejecutando calcular_fecha")
        return datetime.datetime.now().strftime("%Y-%m-%d")

# --- Lógica del Agente ReAct ---

def validar_y_extraer_accion(linea_accion: str) -> Tuple[bool, str, str]:
    """Valida el formato 'herramienta("parametro")' y extrae sus componentes."""
    try:
        # Remover espacios y buscar el patrón herramienta(parametro)
        linea_limpia = linea_accion.strip()
        if '(' not in linea_limpia or ')' not in linea_limpia:
            return False, "", ""

        nombre_herramienta = linea_limpia.split('(')[0].strip()
        if nombre_herramienta not in ['buscar_wikipedia', 'obtener_chiste', 'calcular_fecha']:
            return False, "", ""

        # Extraer parámetro entre paréntesis
        inicio_param = linea_limpia.find('(') + 1
        fin_param = linea_limpia.rfind(')')
        parametro_bruto = linea_limpia[inicio_param:fin_param]
        parametro = parametro_bruto.strip().strip('"\'')

        return True, nombre_herramienta, parametro
    except Exception as e:
        logger.error(f"Error validando acción: {e}")
        return False, "", ""

def ejecutar_accion(nombre_herramienta: str, parametro: str, herramientas: Herramientas) -> str:
    """Ejecuta la herramienta correspondiente y devuelve el resultado."""
    try:
        if nombre_herramienta == 'buscar_wikipedia':
            return herramientas.buscar_wikipedia(parametro)
        elif nombre_herramienta == 'obtener_chiste':
            return herramientas.obtener_chiste()
        elif nombre_herramienta == 'calcular_fecha':
            return herramientas.calcular_fecha()
        return "Error: Herramienta no reconocida."
    except Exception as e:
        logger.error(f"Error ejecutando la herramienta {nombre_herramienta}: {e}")
        return f"Error al ejecutar la herramienta: {e}"

def procesar_respuesta_llm(respuesta_llm: str, herramientas: Herramientas) -> Tuple[str, bool]:
    """
    Procesa la respuesta del LLM. Identifica un 'Thought' y un 'Action',
    ejecuta la acción, y construye un paso ReAct completo con la observación real.
    Rechaza respuestas que empiecen con Final Answer sin usar herramientas.
    """
    paso_actual_str = []
    terminado = False

    thought = None
    action = None
    final_answer = None

    # Parsear la respuesta del LLM línea por línea
    lineas = respuesta_llm.strip().split('\n')

    # Verificar si está intentando dar respuesta final sin usar herramientas
    primera_linea_significativa = None
    for linea in lineas:
        linea_clean = linea.strip()
        if linea_clean:
            primera_linea_significativa = linea_clean.lower()
            break

    # Si empieza con final answer, rechazar y forzar uso de herramientas
    if primera_linea_significativa and (primera_linea_significativa.startswith("final answer:") or primera_linea_significativa.startswith("answer:")):
        return "Thought: No puedo dar una respuesta final sin usar herramientas. Debo buscar la información requerida paso a paso.", False

    for linea in lineas:
        linea = linea.strip()
        if not linea:
            continue

        linea_lower = linea.lower()

        if linea_lower.startswith("thought:"):
            thought = linea
        elif linea_lower.startswith("action:"):
            action = linea
        elif linea_lower.startswith("final answer:") or linea_lower.startswith("answer:"):
            final_answer = linea
            terminado = True
            break

    # Construir el paso de forma controlada
    if thought:
        paso_actual_str.append(thought)

    if action and not terminado:  # Solo ejecutar acción si no hay respuesta final
        paso_actual_str.append(action)
        accion_contenido = action[7:].strip()  # Remover "Action: "
        es_valida, nombre, param = validar_y_extraer_accion(accion_contenido)
        if es_valida:
            observacion = ejecutar_accion(nombre, param, herramientas)
            paso_actual_str.append(f"Observation: {observacion}")
        else:
            paso_actual_str.append("Observation: Error en el formato de la acción. Usa: herramienta(\"parametro\")")

    if final_answer:
        paso_actual_str.append(final_answer)

    # Si no hay contenido válido, forzar el siguiente paso
    if not paso_actual_str:
        return "Thought: Debo usar herramientas para obtener la información solicitada. Empezaré con la primera parte de la pregunta.", False

    return "\n".join(paso_actual_str), terminado

def iniciar_conversacion(query: str, herramientas: Herramientas, max_iteraciones: int = 7) -> str:
    """Gestiona el ciclo completo de la conversación ReAct."""

    system_prompt = """Eres un asistente que DEBE seguir el método ReAct paso a paso. NUNCA puedes responder sin usar herramientas.

HERRAMIENTAS DISPONIBLES:
- buscar_wikipedia("término"): Busca información en Wikipedia en español
- obtener_chiste(): Obtiene un chiste aleatorio en español
- calcular_fecha(): Devuelve la fecha actual

REGLAS ABSOLUTAS - NO NEGOCIABLES:
1. PROHIBIDO inventar información o responder sin herramientas
2. PROHIBIDO dar Final Answer sin haber usado herramientas para TODA la información requerida
3. OBLIGATORIO usar herramientas para cada parte de la pregunta
4. OBLIGATORIO esperar Observation antes de continuar

PROCESO OBLIGATORIO:
1. Identifica QUÉ información necesitas
2. USA herramientas para obtener CADA pieza de información
3. Solo da Final Answer cuando tengas TODAS las Observations necesarias

FORMATO ESTRICTO:
Thought: [Qué necesito hacer ahora]
Action: [herramienta("parametro")]

EJEMPLO PASO A PASO:
Pregunta: "¿Quién fue Einstein y cuéntame un chiste?"

Paso 1:
Thought: Necesito buscar información sobre Einstein. No conozco detalles específicos sin usar herramientas.
Action: buscar_wikipedia("Albert Einstein")

Paso 2 (después de recibir Observation):
Thought: Ya tengo información sobre Einstein. Ahora necesito obtener un chiste usando la herramienta.
Action: obtener_chiste()

Paso 3 (después de recibir segunda Observation):
Thought: Tengo información completa de ambas herramientas. Puedo responder basándome en las Observations recibidas.
Final Answer: [Respuesta usando SOLO información de las Observations]

RESTRICCIONES CRÍTICAS:
- NUNCA digas "René Favaloro fue..." sin haber usado buscar_wikipedia("René Favaloro")
- NUNCA digas un chiste sin haber usado obtener_chiste()
- NUNCA combines información sin haber obtenido cada parte con herramientas
- SIEMPRE usa herramientas ANTES de responder

TU PRIMER PASO DEBE SER SIEMPRE UN "Action:" - NO PUEDES EMPEZAR CON "Final Answer"."""

    user_prompt = f"Pregunta del usuario: {query}"

    mensajes = [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': user_prompt}
    ]

    historial_completo = [user_prompt]
    iteracion = 0

    while iteracion < max_iteraciones:
        iteracion += 1
        logger.info(f"--- Iteración {iteracion} ---")

        try:
            response = ollama.chat(model=LLM_MODEL, messages=mensajes, stream=False)
            respuesta_llm = response['message']['content']

            logger.info(f"Respuesta LLM:\n{respuesta_llm}")

            # Procesar la respuesta y ejecutar la acción
            paso_procesado, terminado = procesar_respuesta_llm(respuesta_llm, herramientas)

            historial_completo.append(paso_procesado)

            # Añadir la respuesta procesada al historial para la siguiente iteración
            mensajes.append({'role': 'assistant', 'content': paso_procesado})

            if terminado:
                logger.info("Conversación terminada (Final Answer encontrada).")
                break

            # Si no terminó, enviar mensaje específico para continuar
            if not terminado:
                mensajes.append({'role': 'user', 'content': 'Continúa con el siguiente paso del proceso ReAct. ¿Qué Action necesitas realizar ahora?'})

        except Exception as e:
            logger.error(f"Error en iteración {iteracion}: {e}")
            break

    if iteracion >= max_iteraciones:
        historial_completo.append("Final Answer: Se alcanzó el límite máximo de iteraciones sin completar la tarea.")

    return "\n\n".join(historial_completo)

# --- Ejecución del Ejemplo ---
if __name__ == "__main__":
    herramientas_agente = Herramientas()

    query_ejemplo = "¿Quién fue René Favaloro y cuéntame un chiste?"
    # query_ejemplo = "¿Qué día es hoy y quién fue Albert Einstein?"

    respuesta_final = iniciar_conversacion(query_ejemplo, herramientas_agente)

    print("\n" + "="*60)
    print("=== CONVERSACIÓN COMPLETA RECONSTRUIDA ===")
    print("="*60)
    print(respuesta_final)


=== CONVERSACIÓN COMPLETA RECONSTRUIDA ===
Pregunta del usuario: ¿Quién fue René Favaloro y cuéntame un chiste?

Thought: Necesito buscar información sobre René Favaloro para entender quién fue.
Action: buscar_wikipedia("René Favaloro")
Observation: René Gerónimo Favaloro (La Plata, 12 de julio de 1923-Buenos Aires, 29 de julio de 2000) fue un médico, inventor, educador y cardiocirujano argentino, reconocido mundialmente por haber desarrollado el baipás coronario con empleo de la vena safena magna.[2]​[3]​[4]​
Estudió Medicina en la Universidad Nacional de La Plata, donde se doctoró con una tesis sobre el íleo.[5]​ Una vez recibido, previo paso por el Hospital Policlínico, se mudó a Jacinto Arauz, provincia de La Pampa, para reemplazar por un tiempo al médico local, quien tenía problemas de salud.[6]​
A su vez, leía bibliografía médica actualizada y empezó a tener interés en la cirugía torácica. A fines de la década de 1960, en la clínica de Cleveland, Ohio (Estados Unidos), comenzó a

## ReAct con Llamaindex

In [ ]:
%%capture
!pip install llama-index-llms-ollama llama-index wikipedia

In [ ]:
%%capture
!ollama pull LLM_MODEL > ollama.log

!pip install litellm[proxy]
!nohup litellm --model ollama/$LLM_MODEL --port 8000 > litellm.log 2>&1 &

In [ ]:
import datetime
import wikipedia
import requests
import logging

from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool
from llama_index.core.agent.react.formatter import ReActChatFormatter

# Configurar el LLM de Ollama
llm = Ollama(
    model=LLM_MODEL,
    request_timeout=120.0, # Aumentado el timeout para modelos más lentos
    temperature=0.0,      # Temperatura en 0 para máxima predictibilidad
    context_window=4096
)
Settings.llm = llm

# --- Definición de Herramientas ---

def obtener_fecha(**kwargs) -> str:
    """Utiliza esta herramienta para obtener la fecha actual. No requiere parámetros."""
    return datetime.datetime.now().strftime("%Y-%m-%d")

def calcular_suma(a: float, b: float) -> str:
    """Utiliza esta herramienta para sumar dos números. Requiere los parámetros 'a' y 'b'."""
    return str(float(a) + float(b))

def buscar_wikipedia(query: str) -> str:
    """Utiliza esta herramienta para buscar un resumen sobre cualquier tema en Wikipedia en español. Requiere un 'query' de búsqueda."""
    try:
        wikipedia.set_lang('es')
        result = wikipedia.summary(query, sentences=3, auto_suggest=False)
        return result
    except wikipedia.exceptions.PageError:
        return f"Error: No se encontró la página para '{query}' en Wikipedia. Intenta con otro término de búsqueda."
    except wikipedia.exceptions.DisambiguationError as e:
        return f"Error: La búsqueda '{query}' es ambigua. Prueba con una de estas opciones: {e.options[:5]}"
    except Exception as e:
        return f"Error: Ocurrió un error inesperado al buscar en Wikipedia: {e}"

def obtener_chiste(**kwargs) -> str:
    """Utiliza esta herramienta para obtener un chiste al azar en español. No requiere parámetros."""
    try:
        # URL de una API que provee chistes en español
        response = requests.get("https://v2.jokeapi.dev/joke/Any?lang=es&type=single,twopart")
        response.raise_for_status()
        joke_data = response.json()

        if joke_data.get("error"):
             return "Error: La API de chistes no pudo encontrar un chiste en español."

        # La API puede devolver chistes de una o dos partes
        if joke_data["type"] == "twopart":
            return f"El chiste obtenido es: {joke_data['setup']} ... {joke_data['delivery']}"
        else: # 'single'
            return f"El chiste obtenido es: {joke_data['joke']}"

    except requests.exceptions.RequestException as e:
        return f"Error: No se pudo contactar a la API de chistes: {e}"
    except Exception as e:
        return f"Error: Ocurrió un error inesperado al obtener el chiste: {e}"

# --- Creación del Agente ---

tools = [
    FunctionTool.from_defaults(fn=obtener_fecha),
    FunctionTool.from_defaults(fn=calcular_suma),
    FunctionTool.from_defaults(fn=buscar_wikipedia),
    FunctionTool.from_defaults(fn=obtener_chiste),
]

# Prompt del sistema mucho más estricto para guiar mejor a los modelos más pequeños.
SYSTEM_PROMPT = """
Eres un asistente que responde en español y sigue reglas de manera muy estricta. Tu objetivo es responder a las preguntas del usuario utilizando un conjunto de herramientas disponibles.

**REGLAS OBLIGATORIAS:**

1.  **PROCESO `Thought -> Action -> Action Input -> Observation`**: Debes seguir este ciclo. NO PUEDES dar una `Final Answer` hasta que hayas recogido TODA la información necesaria.
2.  **UNA HERRAMIENTA A LA VEZ**: En cada paso, solo puedes usar UNA herramienta.
3.  **PREGUNTAS COMPUESTAS**: Si la pregunta del usuario tiene varias partes (p. ej., "busca X y dime la fecha"), DEBES usar las herramientas una por una.
    - Primero, `Thought` sobre la primera parte y usa la herramienta.
    - Después de la `Observation`, `Thought` sobre la segunda parte y usa la otra herramienta.
    - SOLO cuando tengas todas las observaciones, podrás generar la `Final Answer`.
4.  **FORMATO `Action Input`**: El `Action Input` DEBE SER SIEMPRE un JSON.
    - Si la herramienta requiere argumentos (como `buscar_wikipedia`), el formato es `{"query": "término a buscar"}` o `{"a": 1, "b": 2}`.
    - Si la herramienta NO requiere argumentos (como `obtener_fecha`), el formato es OBLIGATORIAMENTE `{}`.
5.  **NO INVENTES RESPUESTAS**: Si la información puede ser obtenida por una herramienta, DEBES usar la herramienta. No inventes resúmenes de Wikipedia, chistes, fechas o cálculos. Usa las herramientas provistas.
6.  **VERIFICACIÓN FINAL**: Antes de dar la `Final Answer`, revisa la pregunta original del usuario y asegúrate de haber respondido a TODAS las partes.

**EJEMPLO DE PREGUNTA COMPUESTA:**
User: Quién fue Albert Einstein y qué día es hoy?

Thought: El usuario tiene dos preguntas. Primero, buscaré información sobre Albert Einstein en Wikipedia.
Action: buscar_wikipedia
Action Input: {"query": "Albert Einstein"}
Observation: [Resumen de Wikipedia sobre Einstein]
Thought: Ya tengo la información sobre Einstein. Ahora necesito obtener la fecha actual para responder la segunda parte de la pregunta.
Action: obtener_fecha
Action Input: {}
Observation: 2025-06-10
Thought: Ahora tengo toda la información necesaria para responder a ambas preguntas. Combinaré los resultados.
Final Answer: Según Wikipedia, Albert Einstein fue un físico teórico... Además, la fecha de hoy es 10 de junio de 2025.
"""

agent = ReActAgent.from_tools(
    tools,
    llm=llm,
    verbose=True,
    chat_formatter=ReActChatFormatter(),
    system_prompt=SYSTEM_PROMPT,
    max_iterations=15, # Damos más margen para preguntas complejas
)

# --- Funciones de Interacción ---

def chat_con_agente(query: str):
    """Función principal para interactuar con el agente ReAct."""
    try:
        if not query or not query.strip():
            return "La consulta no puede estar vacía. Por favor, escribe una pregunta."
        response = agent.chat(query)
        return response
    except Exception as e:
        logging.error(f"Error al procesar la consulta: {e}", exc_info=True)
        return f"Se produjo un error inesperado al procesar tu consulta: {e}"

def ejecutar_ejemplos():
    """Ejecuta una serie de consultas de ejemplo para probar el agente."""
    print(f"=== Iniciando interacción con el Agente ReAct (Modelo: {LLM_MODEL}) ===")

    queries = [
        "¿Puedes contarme un chiste?",
        "¿Quién fue René Favaloro según Wikipedia?",
        "¿Qué día es hoy y cuánto es 512 + 1024?",
        "Busca en Wikipedia qué es un 'agujero negro' y buscar la relación con un chiste",
    ]

    for i, query in enumerate(queries, 1):
        print("\n" + "="*50)
        print(f"|| Consulta {i}: {query}")
        print("="*50)
        response = chat_con_agente(query)
        print(f"\nRespuesta Final del Agente:\n{response}")
        print("-"*(len(str(response))+20))


if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    ejecutar_ejemplos()


=== Iniciando interacción con el Agente ReAct (Modelo: gemma3:12b-it-qat) ===

|| Consulta 1: ¿Puedes contarme un chiste?
> Running step 2e32c03e-b176-4f98-9788-91c0a5f97305. Step input: ¿Puedes contarme un chiste?
Thought: The current language of the user is: Spanish. I need to use a tool to help me answer the question.
Action: obtener_chiste
Action Input: {}
Observation: El chiste obtenido es: No te despedirán del trabajo, si nunca comentas tu código y además eres el único que sabe cómo funciona
> Running step 1aa9f2d8-700d-4a0e-95c2-9a1197959768. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: ¡Aquí tienes el chiste: No te despedirán del trabajo, si nunca comentas tu código y además eres el único que sabe cómo funciona!

Respuesta Final del Agente:
¡Aquí tienes el chiste: No te despedirán del trabajo, si nunca comentas tu código y además eres el único que sabe cómo funciona!
-----------------------------------------

Otros ejemplos:


*   https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/agent/react_agent.ipynb

